In [1]:
import pandas as pd
from sqlalchemy import create_engine

# 1. DB 연결
db_uri = "postgresql+psycopg2://xodh3@localhost:5432/economy_db"
engine = create_engine(db_uri)

# 2. 모든 심볼 조회 쿼리
# market_price_daily 테이블에 있는 고유한(distinct) 심볼만 가져옵니다.
query = "SELECT DISTINCT symbol FROM market_price_daily ORDER BY symbol;"

try:
    with engine.connect() as conn:
        df_symbols = pd.read_sql(query, conn)

    print("📂 DB에 저장된 심볼 목록 (총 {}개):".format(len(df_symbols)))
    print("-" * 30)

    # 보기 좋게 10개씩 끊어서 출력
    symbols = df_symbols['symbol'].tolist()
    for i in range(0, len(symbols), 10):
        print(symbols[i:i+10])

    print("-" * 30)
    print("👉 위 목록에서 '달러(Dollar/USD)'랑 '구리(Copper/Metal)'처럼 생긴 걸 찾아보세요!")

except Exception as e:
    print(f"❌ 오류 발생: {e}")

📂 DB에 저장된 심볼 목록 (총 40개):
------------------------------
['AUDUSD', 'CL1', 'DBA', 'DGS10', 'DGS2', 'DIA', 'DXY', 'EEM', 'EFA', 'EURUSD']
['GBPUSD', 'GLD', 'HYG', 'IEF', 'INDU', 'IWM', 'LQD', 'NASDAQCOM', 'QQQ', 'SLV']
['SPX', 'SPY', 'T10Y2Y', 'TLT', 'US100', 'USDCAD', 'USDCHF', 'USDJPY', 'USDKRW', 'USDKRW-2']
['USGG10YR', 'USO', 'VEA', 'VIXCLS', 'VTI', 'XLE', 'XLF', 'XLK', 'XLRE', 'XLV']
------------------------------
👉 위 목록에서 '달러(Dollar/USD)'랑 '구리(Copper/Metal)'처럼 생긴 걸 찾아보세요!


In [2]:
import pandas as pd
from sqlalchemy import create_engine

# 1. DB 연결
db_uri = "postgresql+psycopg2://xodh3@localhost:5432/economy_db"
engine = create_engine(db_uri)

print("📂 데이터 창고 개방 및 분류 작업 시작...")

# ---------------------------------------------------------
# 1️⃣ 매크로 지표 (Macro Indicators) 가져오기
# ---------------------------------------------------------
query_macro = "SELECT indicator_symbol, date_time, value FROM macro_time_series ORDER BY date_time ASC"
with engine.connect() as conn:
    df_macro_raw = pd.read_sql(query_macro, conn)

# 매크로 정보 요약
macro_summary = []
if not df_macro_raw.empty:
    for symbol in df_macro_raw['indicator_symbol'].unique():
        sub_df = df_macro_raw[df_macro_raw['indicator_symbol'] == symbol]
        last_date = sub_df['date_time'].iloc[-1]
        last_val = sub_df['value'].iloc[-1]

        # 이름 매핑 (알기 쉽게)
        name_map = {
            'FEDFUNDS': '기준금리 (Fed Rate)', 'CPIAUCSL': '물가 (CPI)',
            'RSAFS': '소비 (Retail Sales)', 'UNRATE': '실업률 (Unemployment)',
            'HOUST': '주택착공 (Housing Starts)', 'CSUSHPISA': '집값 (Home Prices)'
        }
        name = name_map.get(symbol, symbol)

        # 카테고리 분류
        category = "매크로(경기)"
        if symbol in ['HOUST', 'CSUSHPISA']: category = "매크로(부동산)"

        macro_summary.append({
            'Category': category,
            'Symbol': symbol,
            'Name': name,
            'Last_Date': last_date.strftime('%Y-%m-%d'),
            'Last_Value': last_val,
            'Trend': '데이터 확인용'
        })

df_macro_report = pd.DataFrame(macro_summary)

# ---------------------------------------------------------
# 2️⃣ 시장 자산 (Market Assets) 가져오기 & 달러 상관관계 분석
# ---------------------------------------------------------
query_market = "SELECT trade_date, symbol, close_price FROM market_price_daily ORDER BY trade_date ASC"
with engine.connect() as conn:
    df_market_raw = pd.read_sql(query_market, conn)

# 피벗팅 (날짜 x 종목)
df_prices = df_market_raw.pivot(index='trade_date', columns='symbol', values='close_price')
df_prices.index = pd.to_datetime(df_prices.index, utc=True)
df_monthly = df_prices.resample('M').last()

# 달러(DXY)와의 상관관계 계산 (최근 6개월)
# DXY가 없으면 계산 불가하므로 체크
dxy_sym = 'DXY'
corrs = {}
if dxy_sym in df_monthly.columns:
    rolling_corr = df_monthly.rolling(6).corr(df_monthly[dxy_sym])
    corrs = rolling_corr.iloc[-1].to_dict()

# 자산 분류 로직
asset_summary = []
for symbol in df_monthly.columns:
    # 1. 최근 가격
    last_price = df_monthly[symbol].iloc[-1]
    last_date = df_monthly.index[-1].strftime('%Y-%m-%d')

    # 2. 상관관계 (달러랑 친구냐 적이냐)
    corr_val = corrs.get(symbol, 0)

    # 3. 자동 카테고리 분류 (규칙 기반)
    # 님 DB에 있는 심볼들을 예상해서 분류합니다.
    category = "기타"
    role = "중립"

    # [공격수] 주식, 코인
    if symbol in ['QQQ', 'SPY', 'DIA', 'VTI', 'IWM', 'XLK', 'NVDA', 'TSLA']:
        category = "⚔️ 공격수 (주식/성장)"
        role = "위험자산"
    elif 'BTC' in symbol or 'ETH' in symbol:
        category = "⚔️ 공격수 (코인)"
        role = "초위험자산"

    # [수비수] 채권, 금, 달러
    elif symbol in ['TLT', 'IEF', 'SHY', 'AGG']:
        category = "🛡️ 수비수 (채권)"
        role = "안전 자산?"
    elif symbol in ['GLD', 'SLV', 'IAU']:
        category = "🛡️ 수비수 (원자재)"
        role = "인플레 방어"
    elif symbol in ['DXY', 'UUP', 'USDKRW', 'USDJPY']:
        category = "🛡️ 수비수 (현금/환율)"
        role = "최종 병기"

    # [실물/경기] 에너지, 산업재, 구리
    elif symbol in ['USO', 'CL1', 'XLE']:
        category = "🏭 실물 (에너지)"
        role = "인플레 연동"
    elif symbol in ['CPER', 'DBA']:
        category = "🏭 실물 (원자재)"

    # [심리]
    elif symbol in ['VIX', 'VIXCLS']:
        category = "👻 심리 (공포)"
        role = "역상관"

    # 상관관계에 따른 역할 재정의 (데이터 기반)
    if category == "기타":
        if corr_val < -0.5: category = "⚔️ 공격수 (달러 역행)"
        elif corr_val > 0.5: category = "🛡️ 수비수 (달러 동행)"

    asset_summary.append({
        'Category': category,
        'Symbol': symbol,
        'Last_Price': f"{last_price:.2f}",
        'Corr_with_DXY': f"{corr_val:.2f}",
        'Role_Analysis': f"{'달러의 적(Risk)' if corr_val < -0.3 else '달러의 친구(Hedge)' if corr_val > 0.3 else '독자 행동'}"
    })

df_asset_report = pd.DataFrame(asset_summary).sort_values(by=['Category', 'Symbol'])

# ---------------------------------------------------------
# 📊 최종 리포트 출력
# ---------------------------------------------------------
print("\n" + "="*60)
print("🌍 [Part 1] 경제 환경 (Macro Environment)")
print("="*60)
print(df_macro_report[['Category', 'Name', 'Last_Value', 'Last_Date']].to_string(index=False))

print("\n" + "="*60)
print("💰 [Part 2] 내 자산 분류표 (Portfolio Classification)")
print("="*60)
# 보기 좋게 컬럼 순서 조정
cols = ['Category', 'Symbol', 'Last_Price', 'Corr_with_DXY', 'Role_Analysis']
print(df_asset_report[cols].to_string(index=False))

print("\n💡 팁: 'Corr_with_DXY'가 양수(+)면 달러 오를 때 같이 오르는 놈, 음수(-)면 떨어지는 놈입니다.")

📂 데이터 창고 개방 및 분류 작업 시작...

🌍 [Part 1] 경제 환경 (Macro Environment)
Category                                                  Name  Last_Value  Last_Date
 매크로(경기)                     UNITED_STATES_CORPORATE_TAX_RATE_          21 2025-12-31
 매크로(경기)                      UNITED_STATES_CORPORATE_TAX_RATE          21 2025-12-31
 매크로(경기)                         UNITED_STATES_INFLATION_RATE_           3 2025-08-31
 매크로(경기)                          UNITED_STATES_INFLATION_RATE           3 2025-08-31
 매크로(경기)                                                INDPRO         104 2025-08-01
 매크로(경기)                    UNITED_STATES_CRUDE_OIL_PRODUCTION       13580 2025-06-30
 매크로(경기)                   UNITED_STATES_INDUSTRIAL_PRODUCTION           1 2025-08-31
 매크로(경기)                   UNITED_STATES_CRUDE_OIL_PRODUCTION_       13580 2025-06-30
 매크로(경기)                  UNITED_STATES_INDUSTRIAL_PRODUCTION_           1 2025-08-31
 매크로(경기)                                                PAYEMS      159540 2

/var/folders/px/mxxc21bs63gcw44m35sp_51r0000gn/T/ipykernel_3027/1711499766.py:58: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly = df_prices.resample('M').last()


In [3]:
import pandas as pd
from sqlalchemy import create_engine

# 1. DB 연결
db_uri = "postgresql+psycopg2://xodh3@localhost:5432/economy_db"
engine = create_engine(db_uri)

print("🚑 데이터 응급실 가동 중...", end=" ")

# 2. 모든 데이터 가져오기
query = "SELECT trade_date, symbol, close_price FROM market_price_daily ORDER BY trade_date ASC"
with engine.connect() as conn:
    df_raw = pd.read_sql(query, conn)

# 피벗팅 (날짜 x 종목)
df_all = df_raw.pivot(index='trade_date', columns='symbol', values='close_price')
df_all.index = pd.to_datetime(df_all.index, utc=True)

print("완료! ✅")

# ---------------------------------------------------------
# 🏥 1단계: 환자 상태 확인 (진단)
# ---------------------------------------------------------
# 각 종목별로 데이터가 언제 끊겼는지 확인
last_dates = df_all.apply(lambda x: x.last_valid_index())
last_dates_str = last_dates.dt.strftime('%Y-%m-%d')

print("\n📋 [진단 리포트] 각 종목의 마지막 데이터 날짜:")
# 문제가 되었던 종목들(nan 떴던 애들) 위주로 확인
problem_kids = ['DIA', 'IWM', 'VTI', 'XLK', 'HYG', 'QQQ', 'DXY']
for sym in problem_kids:
    if sym in last_dates_str:
        print(f"  - {sym}: {last_dates_str[sym]}")
    else:
        print(f"  - {sym}: ❌ 데이터 없음 (DB에 아예 없음)")

# ---------------------------------------------------------
# 💊 2단계: 심폐소생술 (Forward Fill)
# ---------------------------------------------------------
# 주말/휴일 때문에 비어있는 곳을 '전날 데이터'로 채웁니다.
# (예: 금요일 가격을 토, 일요일에도 유지)
df_filled = df_all.ffill()

# 월말 데이터로 리샘플링 (이제 nan이 없어야 함)
df_monthly = df_filled.resample('M').last()

# ---------------------------------------------------------
# 📊 3단계: 되살아난 데이터로 재분석
# ---------------------------------------------------------
target = 'DXY'

if target in df_monthly.columns:
    # 상관관계 다시 계산
    corrs = df_monthly.rolling(6).corr(df_monthly[target]).iloc[-1]

    # 결과 정리
    revived_summary = []
    for sym in problem_kids:
        if sym in corrs:
            corr_val = corrs[sym]
            status = "✅ 소생 성공" if pd.notna(corr_val) else "💀 소생 실패"
            revived_summary.append({
                'Symbol': sym,
                'Corr_with_DXY': f"{corr_val:.2f}",
                'Status': status
            })

    print("\n✨ [수술 결과] 데이터 소생 여부 확인:")
    print(pd.DataFrame(revived_summary))

    # 전체 랭킹 다시 보여주기 (ETF만)
    print("\n🏆 [Final] 되살아난 ETF 달러 민감도 랭킹:")
    rank = corrs.drop(target, errors='ignore').dropna().sort_values()

    # 보기 좋게 상위/하위 5개
    print("Top 3 (달러랑 반대 - 위험):")
    print(rank.head(3))
    print("\nTop 3 (달러랑 친구 - 안전):")
    print(rank.tail(3))

else:
    print("❌ DXY 데이터가 없어서 분석 불가")

🚑 데이터 응급실 가동 중... 완료! ✅

📋 [진단 리포트] 각 종목의 마지막 데이터 날짜:
  - DIA: 2025-09-16
  - IWM: 2025-09-16
  - VTI: 2025-09-16
  - XLK: 2025-09-16
  - HYG: 2025-09-16
  - QQQ: 2025-12-10
  - DXY: 2025-12-08

✨ [수술 결과] 데이터 소생 여부 확인:
  Symbol Corr_with_DXY   Status
0    DIA         -0.45  ✅ 소생 성공
1    IWM         -0.45  ✅ 소생 성공
2    VTI         -0.48  ✅ 소생 성공
3    XLK         -0.44  ✅ 소생 성공
4    HYG         -0.47  ✅ 소생 성공
5    QQQ         -0.32  ✅ 소생 성공
6    DXY          1.00  ✅ 소생 성공

🏆 [Final] 되살아난 ETF 달러 민감도 랭킹:
Top 3 (달러랑 반대 - 위험):
symbol
USGG10YR   -0.903315
CL1        -0.854235
VTI        -0.475725
Name: 2025-12-31 00:00:00+00:00, dtype: float64

Top 3 (달러랑 친구 - 안전):
symbol
DGS10       0.475348
USDKRW      0.907742
USDKRW-2    0.907742
Name: 2025-12-31 00:00:00+00:00, dtype: float64


/var/folders/px/mxxc21bs63gcw44m35sp_51r0000gn/T/ipykernel_3027/2704961811.py:45: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_monthly = df_filled.resample('M').last()
